In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the subject's DataFrame
big_df = pd.read_csv(r"W:\Projects\2019-04 M1M1PAS Project\analysis\source_Paolo\fc_source\sourceConnectivity.csv",index_col=0)

# Print the reshaped DataFrame
print(big_df)

In [ ]:
# Fix variable types and check for NaNs
big_df['run'] = big_df['run'].astype(str)
print(big_df.dtypes)
print(big_df.isna().sum())

In [ ]:
# Group by the desired columns and calculate median
con_mes = 'aec'

summary_df_ind = big_df.groupby(['run', 'task', 'subject'], as_index=False).agg(
    median_sub_connect=(con_mes, 'median'),
)
print(summary_df_ind)

In [ ]:
# Function to calculate 25th and 75th percentiles
def q25(x):
    return np.percentile(x, 25)

def q75(x):
    return np.percentile(x, 75)

# Group by 'run', 'task' and calculate medians and quartiles
summary_df = summary_df_ind.groupby(['run', 'task']).agg(
    median_connect=('median_sub_connect', 'median'),
    q1_connect=('median_sub_connect', q25),
    q3_connect=('median_sub_connect', q75)
).reset_index()

# Calculate the errors (distance between median and quartiles)
summary_df['lower_err'] = summary_df['median_connect'] - summary_df['q1_connect']
summary_df['upper_err'] = summary_df['q3_connect'] - summary_df['median_connect']

In [ ]:
data_to_plot = summary_df

# Plot
plt.figure(figsize=(10, 6))

# Plot the median values using seaborn
sns.lineplot(
    data=data_to_plot, 
    x='run', 
    y='median_connect', 
    hue='task', 
    marker='o', 
    ci=None  # We will add custom error bars for quartiles
)

# Get the colors used by seaborn for each task line
palette = sns.color_palette()

# Add custom error bars for the quartiles with the same colors as the lines
for i, task in enumerate(data_to_plot['task'].unique()):
    task_data = data_to_plot[data_to_plot['task'] == task]
    plt.errorbar(
        task_data['run'], 
        task_data['median_connect'], 
        yerr=[task_data['lower_err'], task_data['upper_err']], 
        fmt='none',  # No line, just error bars
        capsize=5,  # Optional: adds caps to the error bars
        color=palette[i],  # Apply the same color as the line
    )

# Customize the plot
plt.title("Median AEC with Quartiles")
plt.xlabel("Run")
plt.ylabel("Median AEC")
plt.legend(title="Task")
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
# Get the unique tasks for this subject
tasks = summary_df_ind['task'].unique()
    
# Create a figure with subplots for each task
fig, axes = plt.subplots(nrows=1, ncols=len(tasks), figsize=(15, 5), sharey=True)
fig.suptitle(f'Subject-median Connectivity', fontsize=16)

    
# Iterate through tasks and create a violin plot for each
for i, task in enumerate(tasks):
    task_df = summary_df_ind[summary_df_ind['task'] == task]
    sns.violinplot(ax=axes[i], x='run', y='median_sub_connect', data=task_df)
    axes[i].set_title(f'Task: {task}')
    axes[i].set_xlabel('Run')
    axes[i].set_ylabel('Connectivity subject-median Value')

# Show the plot for this subject
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()